# 배경 이미지 생성

여기서는 모델 학습에 사용할 배경 이미지를 생성합니다.

아스팔트나 콘크리트 바닥같은 아무 사진이나 구해오면 되지만 그건 귀찮고,

더 편한 방법이 있으므로 그냥 대충 만듭니다.

목표는 400장인데 약간 더 많이 만들고 눈으로 골라내겠습니다.


## 방법

1. 바운딩 박스가 2개 이상이면 그냥 버립니다.
2. 바운딩 박스의 왼/오른쪽은 너비, 위/아래쪽은 높이가 됩니다.
3. 일단은 둘 모두 320 이상이 될 수 있는 이미지가 충분히 있는지를 보려고 합니다.
4. 그게 장수가 충분히 많다면 거기서 고르고, 너무 적으면 기준을 좀 낮춥니다.
5. json에서 읽어도 되지만 yolov5 라벨도 만들었으므로 그걸로 하는 게 좋을 것 같습니다.



근데 생각해보니 기왕 이렇게 할거면 train에서 배경을 뽑아오는 것 보다 학습 동안 어차피
안 쓸 validation 셋에서 뽑아보는 것도 좋을 것 같습니다.

숫자만 맞으면 됩니다.


In [11]:
import os
import random

# from collections import namedtuple
from pathlib import Path
from pprint import pprint
from typing import (
    Any, NamedTuple, NewType, Optional, Sequence, Tuple, Union
)

from PIL import Image


random.seed()



In [15]:
Rect = NewType('Rect', Sequence[Tuple[int, int]])

class Label(object):
    '''yolov5 라벨 한 줄(=바운딩 박스 1개)을 읽어 해석합니다.'''
    classid: int
    centerx: int
    centery: int
    width: int
    height: int
    dim: int

    def __init__(self, text: str, dim: int = 640) -> None:
        self.dim = int(dim)

        cid, cx, cy, w, h = map(float, text.strip().split(' '))
        self.classid = cid
        self.centerx = round(cx * self.dim)
        self.centery = round(cy * self.dim)
        self.width = round(w * self.dim)
        self.height = round(h * self.dim)

    def biggest_rect(self, threshold: int) -> Optional[Rect]:
        '''바운딩 박스를 제외한 가장 큰 크랍 렉트를 반환합니다.
        
        가장 큰 렉트가 ``threshold``보다 작으면 ``None``을 반환합니다.
        '''
        edges = [
            self.centerx - self.width//2,
            self.dim - self.centerx - self.width//2,
            self.centery - self.height//2,
            self.dim - self.centery - self.height//2,
        ]
        limiting_edge = max(edges)
        if limiting_edge < threshold:
            return None
            
        edge_index = edges.index(limiting_edge)


In [13]:
def nominate(top: Path, min_dim: float = 0.5) -> Sequence[Tuple[Path, Rect]]:
    cache = set()
    for pathname in top.rglob('*.txt'):
        with open(pathname, 'r', encoding='utf-8') as txt_in:
            lines = txt_in.readlines()
            if len(lines) > 1:
                # 2줄 이상인 파일은 곧바로 무시합니다.
                continue

            words = lines[0].strip().split(' ')
            label = Label(
                int(words[0]),
                float(words[1]),
                float(words[2]),
                float(words[3]),
                float(words[4]),
            )

SyntaxError: invalid syntax (1444649845.py, line 10)